In [3]:
!pip install opencv-python

In [7]:
!pip install pyttsx3

In [2]:
import numpy as np
import cv2
import mediapipe as mp
import pyttsx3

print("All required modules are installed successfully!")

All required modules are installed successfully!


In [ ]:
#adding new eexercises
"push_up": {
    "landmarks": ["shoulder", "elbow", "wrist"],
    "angles": {
        "elbow": {"min": 90, "max": 160, "feedback": ["Bend your elbows to 90 degrees.", "Don't lock your elbows."]}
    }
}


In [24]:
#works the best so faaaaar
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
from datetime import datetime
import random

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# Video capture
cap = cv2.VideoCapture(0)

# Initialize counters, stages, and game variables
counter = 0
stage = None
feedback = ""
current_mode = "workout"  # Default mode: "workout" or "boxing"
current_exercise = "squat"
score = 0
target_position = (random.randint(100, 500), random.randint(100, 400))
target_radius = 50

# Mediapipe Pose and Hands setup
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
        mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        if not ret:
            print("Error: Frame not captured.")
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process detections
        pose_results = pose.process(image)
        hand_results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Mode Logic
        if current_mode == "workout":
            # Exercise logic
            try:
                landmarks = pose_results.pose_landmarks.landmark
                if current_exercise == "squat":
                    # Squat logic
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    angle = calculate_angle(hip, knee, ankle)
                    if angle > 160:
                        stage = "up"
                    if angle < 90 and stage == "up":
                        stage = "down"
                        counter += 1
                        engine.say("Good job! Keep going.")
                        engine.runAndWait()

                    # Feedback for posture
                    if angle < 70:
                        feedback = "Bend your knees more."
                        engine.say(feedback)
                        engine.runAndWait()

                elif current_exercise == "plank":
                    # Add plank logic here
                    pass

                elif current_exercise == "warrior_pose":
                    # Add yoga warrior pose logic here
                    pass

            except Exception as e:
                feedback = "Pose not detected. Adjust yourself."

            # Render feedback
            cv2.putText(image, f'Exercise: {current_exercise.upper()}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Reps: {counter}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        elif current_mode == "boxing":
            # Boxing game logic
            cv2.circle(image, target_position, target_radius, (0, 255, 0), -1)
            cv2.putText(image, f'Score: {score}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Detect hand hits
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame.shape[1])
                    wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame.shape[0])

                    if calculate_distance((wrist_x, wrist_y), target_position) < target_radius:
                        score += 1
                        target_position = (random.randint(100, frame.shape[1] - 100),
                                           random.randint(100, frame.shape[0] - 100))
                        engine.say("Hit!")
                        engine.runAndWait()

        # Draw landmarks
        mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the frame
        cv2.imshow("Fitness Assistant", image)

        # Key press handling
        key = cv2.waitKey(10)
        if key == ord('q'):  # Quit
            break
        elif key == ord('1'):
            current_mode = "workout"
            current_exercise = "squat"
            counter, stage = 0, None
        elif key == ord('2'):
            current_mode = "workout"
            current_exercise = "plank"
            counter, stage = 0, None
        elif key == ord('3'):
            current_mode = "workout"
            current_exercise = "warrior_pose"
            counter, stage = 0, None
        elif key == ord('g'):  # Start boxing game
            current_mode = "boxing"
            score = 0

    cap.release()
    cv2.destroyAllWindows()


In [36]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
from datetime import datetime
import random
import time

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# Video capture
cap = cv2.VideoCapture(0)

# Initialize counters, stages, and game variables
counter = 0
stage = None
feedback = ""
current_mode = "workout"  # Default mode: "workout" or "boxing"
current_exercise = "squat"
score = 0
target_position = (random.randint(100, 500), random.randint(100, 400))
target_radius = 50
start_time = datetime.now()
calories = 0.0

# Exercise-specific constants for calorie calculation
CALORIE_CONSTANTS = {
    "squat": 0.32,  # calories per rep
    "plank": 3.5,  # calories per minute
    "lunges": 0.4,  # calories per rep
    "jumping_jacks": 10.0,  # calories per minute
    "warrior_pose": 2.5,  # calories per minute
}

# Mediapipe Pose and Hands setup
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
        mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process detections
        pose_results = pose.process(image)
        hand_results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Get exercise duration
        elapsed_time = (datetime.now() - start_time).total_seconds()
        elapsed_minutes = elapsed_time / 60

        # Mode Logic
        if current_mode == "workout":
            try:
                landmarks = pose_results.pose_landmarks.landmark
                if current_exercise == "squat":
                    # Squat logic
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    angle = calculate_angle(hip, knee, ankle)
                    if angle > 160:
                        stage = "up"
                    if angle < 90 and stage == "up":
                        stage = "down"
                        counter += 1
                        calories += CALORIE_CONSTANTS["squat"]
                        engine.say("Good job! Keep going.")
                        engine.runAndWait()

                    # Feedback for posture
                    if angle < 70:
                        feedback = "Bend your knees more."
                    elif angle > 160:
                        feedback = "Straighten up."

                elif current_exercise == "plank":
                    # Plank logic
                    calories += CALORIE_CONSTANTS["plank"] * elapsed_minutes
                    feedback = "Hold your body straight."

                elif current_exercise == "lunges":
                    # Lunges logic
                    # Similar to squats but with different landmarks
                    pass

                elif current_exercise == "jumping_jacks":
                    # Jumping jacks logic
                    calories += CALORIE_CONSTANTS["jumping_jacks"] * elapsed_minutes
                    feedback = "Keep jumping!"

                elif current_exercise == "warrior_pose":
                    # Warrior pose logic
                    calories += CALORIE_CONSTANTS["warrior_pose"] * elapsed_minutes
                    feedback = "Keep your arms straight."

            except Exception as e:
                feedback = "Pose not detected. Adjust yourself."

            # Display feedback, calories, and counter
            cv2.putText(image, f'Exercise: {current_exercise.upper()}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Reps: {counter}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Calories: {calories:.1f}', (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Feedback: {feedback}', (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

        elif current_mode == "boxing":
            # Boxing game logic
            cv2.circle(image, target_position, target_radius, (0, 255, 0), -1)
            cv2.putText(image, f'Score: {score}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Detect hand hits
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame.shape[1])
                    wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame.shape[0])

                    if calculate_distance((wrist_x, wrist_y), target_position) < target_radius:
                        score += 1
                        target_position = (random.randint(100, frame.shape[1] - 100),
                                           random.randint(100, frame.shape[0] - 100))
                        engine.say("Hit!")
                        engine.runAndWait()

        # Draw landmarks
        mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the frame
        cv2.imshow("Fitness Assistant", image)

        # Key press handling
        key = cv2.waitKey(10)
        if key == ord('q'):  # Quit
            break
        elif key == ord('1'):
            current_mode = "workout"
            current_exercise = "squat"
            counter, stage = 0, None
        elif key == ord('2'):
            current_mode = "workout"
            current_exercise = "plank"
            counter, stage = 0, None
        elif key == ord('3'):
            current_mode = "workout"
            current_exercise = "lunges"
            counter, stage = 0, None
        elif key == ord('4'):
            current_mode = "workout"
            current_exercise = "jumping_jacks"
            counter, stage = 0, None
        elif key == ord('5'):
            current_mode = "workout"
            current_exercise = "warrior_pose"
            counter, stage = 0, None
        elif key == ord('g'):  # Start boxing game
            current_mode = "boxing"
            score = 0

    cap.release()
    cv2.destroyAllWindows()


In [6]:
#if nothign wokrs then this code 
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
from datetime import datetime
import random
import time
import threading

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Initialize text-to-speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Set speech rate for clarity

def async_speech_feedback(text):
    # Non-blocking function for voice feedback
    def speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=speak).start()

# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# Video capture
cap = cv2.VideoCapture(0)

# Initialize counters, stages, and game variables
counter = 0
stage = None
feedback = ""
current_mode = "workout"  # Default mode: "workout" or "boxing"
current_exercise = "squat"
score = 0
target_position = (random.randint(100, 500), random.randint(100, 400))
target_radius = 50
start_time = datetime.now()
calories = 0.0

# Exercise-specific constants for calorie calculation
CALORIE_CONSTANTS = {
    "squat": 0.32,  # calories per rep
    "plank": 3.5,  # calories per minute
    "lunges": 0.4,  # calories per rep
    "jumping_jacks": 10.0,  # calories per minute
    "warrior_pose": 2.5,  # calories per minute
}

# Mediapipe Pose and Hands setup
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
        mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process detections
        pose_results = pose.process(image)
        hand_results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Get exercise duration
        elapsed_time = (datetime.now() - start_time).total_seconds()
        elapsed_minutes = elapsed_time / 60

        # Mode Logic
        if current_mode == "workout":
            try:
                landmarks = pose_results.pose_landmarks.landmark
                if current_exercise == "squat":
                    # Squat logic
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    angle = calculate_angle(hip, knee, ankle)
                    if angle > 160:
                        stage = "up"
                    if angle < 90 and stage == "up":
                        stage = "down"
                        counter += 1
                        calories += CALORIE_CONSTANTS["squat"]
                        async_speech_feedback("Good job! Keep going.")

                    # Feedback for posture
                    if angle < 70:
                        feedback = "Bend your knees more."
                        async_speech_feedback(feedback)
                    elif angle > 160:
                        feedback = "Straighten up."
                        async_speech_feedback(feedback)

                elif current_exercise == "plank":
                    # Plank logic (detect holding the plank position)
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    # Ensure the body is aligned horizontally (plank position)
                    if calculate_angle(shoulder, elbow, wrist) < 170 and calculate_angle(hip, knee, ankle) > 170:
                        feedback = "Good plank position! Keep holding."
                        async_speech_feedback(feedback)
                        calories += CALORIE_CONSTANTS["plank"] * elapsed_minutes
                    else:
                        feedback = "Adjust your position."
                        async_speech_feedback(feedback)

                elif current_exercise == "warrior_pose":
                    # Warrior pose logic (detect arm and leg alignment)
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    # Check for correct leg and arm alignment for warrior pose
                    if calculate_angle(shoulder, elbow, wrist) > 160 and calculate_angle(hip, knee, ankle) < 90:
                        feedback = "Good warrior pose! Keep going."
                        async_speech_feedback(feedback)
                        calories += CALORIE_CONSTANTS["warrior_pose"] * elapsed_minutes
                    else:
                        feedback = "Adjust your arms and legs."
                        async_speech_feedback(feedback)

            except Exception as e:
                feedback = "Pose not detected. Adjust yourself."
                print(f"Error: {e}")

            # Display feedback, calories, and counter
            cv2.putText(image, f'Exercise: {current_exercise.upper()}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Reps: {counter}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Calories: {calories:.1f}', (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Feedback: {feedback}', (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Stage: {stage}', (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

        elif current_mode == "boxing":
            # Boxing game logic
            cv2.circle(image, target_position, target_radius, (0, 255, 0), -1)
            cv2.putText(image, f'Score: {score}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Detect hand hits
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame.shape[1])
                    wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame.shape[0])

                    if calculate_distance((wrist_x, wrist_y), target_position) < target_radius:
                        score += 1
                        target_position = (random.randint(100, frame.shape[1] - 100),
                                           random.randint(100, frame.shape[0] - 100))
                        async_speech_feedback("Hit!")

        # Draw landmarks
        mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the frame
        cv2.imshow("Fitness Assistant", image)

        # Key press handling
        key = cv2.waitKey(10)
        if key == ord('q'):  # Quit
            break
        elif key == ord('1'):
            current_mode = "workout"
            current_exercise = "squat"
            counter, stage = 0, None
        elif key == ord('2'):
            current_mode = "workout"
            current_exercise = "plank"
            counter, stage = 0, None
        elif key == ord('3'):
            current_mode = "workout"
            current_exercise = "lunges"
            counter, stage = 0, None
        elif key == ord('4'):
            current_mode = "workout"
            current_exercise = "jumping_jacks"
            counter, stage = 0, None
        elif key == ord('5'):
            current_mode = "workout"
            current_exercise = "warrior_pose"
            counter, stage = 0, None
        elif key == ord('g'):  # Start boxing game
            current_mode = "boxing"
            score = 0

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
#final code done 
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
from datetime import datetime
import random
import time
import threading

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Initialize text-to-speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Set speech rate for clarity

def async_speech_feedback(text):
    # Non-blocking function for voice feedback
    def speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=speak).start()

# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# Video capture
cap = cv2.VideoCapture(0)

# Initialize counters, stages, and game variables
counter = 0
stage = None
feedback = ""
current_mode = "workout"  # Default mode: "workout" or "boxing"
current_exercise = "squat"
score = 0
target_position = (random.randint(100, 500), random.randint(100, 400))
target_radius = 50
start_time = datetime.now()
calories = 0.0

# Exercise-specific constants for calorie calculation
CALORIE_CONSTANTS = {
    "squat": 0.32,  # calories per rep
    "plank": 3.5,  # calories per minute
    "lunges": 0.4,  # calories per rep
    "jumping_jacks": 10.0,  # calories per minute
    "warrior_pose": 2.5,  # calories per minute
}

# Mediapipe Pose and Hands setup
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, \
        mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not captured.")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process detections
        pose_results = pose.process(image)
        hand_results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Get exercise duration
        elapsed_time = (datetime.now() - start_time).total_seconds()
        elapsed_minutes = elapsed_time / 60

        # Mode Logic
        if current_mode == "workout":
            try:
                landmarks = pose_results.pose_landmarks.landmark
                if current_exercise == "squat":
                    # Squat logic
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    angle = calculate_angle(hip, knee, ankle)
                    if angle > 160:
                        stage = "up"
                    if angle < 90 and stage == "up":
                        stage = "down"
                        counter += 1
                        calories += CALORIE_CONSTANTS["squat"]
                        async_speech_feedback("Good job! Keep going.")

                    # Feedback for posture
                    if angle < 70:
                        feedback = "Bend your knees more."
                        async_speech_feedback(feedback)
                    elif angle > 160:
                        feedback = "Straighten up."
                        async_speech_feedback(feedback)

                elif current_exercise == "plank":
                    # Plank logic (detect holding the plank position)
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    # Ensure the body is aligned horizontally (plank position)
                    if calculate_angle(shoulder, elbow, wrist) < 170 and calculate_angle(hip, knee, ankle) > 170:
                        feedback = "Good plank position! Keep holding."
                        async_speech_feedback(feedback)
                        calories += CALORIE_CONSTANTS["plank"] * elapsed_minutes
                    else:
                        feedback = "Adjust your position."
                        async_speech_feedback(feedback)

                elif current_exercise == "warrior_pose":
                    # Warrior pose logic (detect arm and leg alignment)
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    # Check for correct leg and arm alignment for warrior pose
                    arm_angle = calculate_angle(shoulder, elbow, wrist)
                    leg_angle = calculate_angle(hip, knee, ankle)

                    if arm_angle > 160 and leg_angle < 90:
                        feedback = "Good warrior pose! Keep going."
                        async_speech_feedback(feedback)
                        calories += CALORIE_CONSTANTS["warrior_pose"] * elapsed_minutes
                    else:
                        feedback = "Adjust your arms and legs."
                        async_speech_feedback(feedback)

                elif current_exercise == "lunges":
                    # Lunges logic (track one leg bent and the other straight)
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    # Calculate the knee angle for left leg (bent leg)
                    angle = calculate_angle(hip, knee, ankle)

                    # If the left leg is bent (angle less than 90 degrees), count a lunge
                    if angle < 90 and stage == "up":
                        stage = "down"
                        counter += 1
                        calories += CALORIE_CONSTANTS["lunges"]
                        async_speech_feedback("Good lunge! Keep going.")

                    if angle > 160:
                        feedback = "Straighten your legs."
                        async_speech_feedback(feedback)

                    elif angle < 90:
                        feedback = "Bend your knee more."
                        async_speech_feedback(feedback)

                elif current_exercise == "jumping_jacks":
                    # Jumping jacks logic (arms and legs wide apart)
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                    # Detect wide arm angle
                    arm_angle = calculate_angle(shoulder, elbow, wrist)
                    leg_angle = calculate_angle(hip, knee, ankle)

                    # Arms and legs should be wide open during jumping jacks
                    if arm_angle > 160 and leg_angle > 160:
                        feedback = "Good jumping jack! Keep it up."
                        async_speech_feedback(feedback)
                        calories += CALORIE_CONSTANTS["jumping_jacks"] * elapsed_minutes
                    else:
                        feedback = "Spread your arms and legs wider."
                        async_speech_feedback(feedback)

            except Exception as e:
                feedback = "Pose not detected. Adjust yourself."
                print(f"Error: {e}")

            # Display feedback, calories, and counter
            cv2.putText(image, f'Exercise: {current_exercise.upper()}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Reps: {counter}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Calories: {calories:.1f}', (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Feedback: {feedback}', (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Stage: {stage}', (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

        elif current_mode == "boxing":
            # Boxing game logic
            cv2.circle(image, target_position, target_radius, (0, 255, 0), -1)
            cv2.putText(image, f'Score: {score}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Detect hand hits
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame.shape[1])
                    wrist_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame.shape[0])

                    if calculate_distance((wrist_x, wrist_y), target_position) < target_radius:
                        score += 1
                        target_position = (random.randint(100, frame.shape[1] - 100),
                                           random.randint(100, frame.shape[0] - 100))
                        async_speech_feedback("Hit!")

        # Draw landmarks
        mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the frame
        cv2.imshow("Fitness Assistant", image)

        # Key press handling
        key = cv2.waitKey(10)
        if key == ord('q'):  # Quit
            break
        elif key == ord('1'):
            current_mode = "workout"
            current_exercise = "squat"
            counter, stage = 0, None
        elif key == ord('2'):
            current_mode = "workout"
            current_exercise = "plank"
            counter, stage = 0, None
        elif key == ord('3'):
            current_mode = "workout"
            current_exercise = "lunges"
            counter, stage = 0, None
        elif key == ord('4'):
            current_mode = "workout"
            current_exercise = "jumping_jacks"
            counter, stage = 0, None
        elif key == ord('5'):
            current_mode = "workout"
            current_exercise = "warrior_pose"
            counter, stage = 0, None
        elif key == ord('g'):  # Start boxing game
            current_mode = "boxing"
            score = 0

    cap.release()
    cv2.destroyAllWindows()


C:\Users\deept\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Exception in thread Thread-6 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-7 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaco

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'


Exception in thread Thread-24 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-25 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pyttsx

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'


Exception in thread Thread-39 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


...

Exception in thread Thread-260 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-261 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pytt

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'


Exception in thread Thread-596 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-598 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
Exception in thread Thread-599 (speak):
Traceback (most

Error: 'NoneType' object has no attribute 'landmark'


Exception in thread Thread-603 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-604 (speak):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\deept\AppData\Local\Temp\ipykernel_7196\2015878254.py", line 37, in speak
  File "C:\ProgramData\anaconda3\Lib\site-packages\pytt

###### 